# Pre requisite

In [1]:
from collections import deque
import pandas as pd
import numpy as np
from copy import deepcopy
class Node:
    def __init__(self, data, level=0,index=0):
        self.left = None
        self.index=index
        self.data = data  # Tuple
        self.right =None
        self.parent = None
        self.level = level
        self.best_dis=float('inf')
        self.best_node=None
        self.nearest=set()
        self.k=1


In [50]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 60%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


ioi=1

# KD Tree

In [ ]:
class KDTree:
    def __init__(self, size):
        self.size = size
        self.head=Node([size,size])
        self.root=Node(None)
        self.root.parent=self.head
        self.root.level=0
        self.nodes=deque()
    def generate(self,alls:list):
        t=alls[0]
        counter=1
        self.root.data=t
        self.root.index=0
        alls=iter(alls[1:])
        while(True):
            try:
                d=next(alls)

                self.insertion(d,counter)
                counter+=1
            except:
                break
    def insertion(self,data,counter):
            p=self.root
            level=0
            sw=0
            while(sw==0):
                while(data[level%len(data)]<p.data[level%len(data)]):
                    level+=1
                    if(p.left is not None):
                        p=p.left
                    else:
                        p.left=Node(data,level=level,index=counter)
                        p.left.parent=p
                        sw=1
                        break
                while(data[level%len(data)]>=p.data[level%len(data)] and sw==0):
                    level+=1
                    if(p.right is not None):
                        p=p.right
                    else:
                        p.right=Node(data,level,index=counter)
                        p.right.parent=p
                        sw=1
                        break
    def dist(self, p1, p2):
        return sum([(a - b) ** 2 for a, b in zip(p1, p2)])
    def qualify(self,my_point:Node,examinary_data:Node):
        d=self.dist(my_point.data,examinary_data.data)
        t=(examinary_data,d)
        my_point.nearest.add(t)
        # print(examinary_data)
        if(d<my_point.best_dis):
            my_point.best_dis=d
            my_point.best_node=examinary_data
    def gate_check(self,root:Node):
        #1 leaf
        #2 right is ok
        #3 left is ok

        if(root.left is None and root.right is None):
            return 1
        elif(root.left is not None):
            return 3
        elif(root.right is not None):
            return 2
    def search(self,data:Node,root:Node,level):
        tb=0
        # print(root)
        # print(root.data)
        # print(self.gate_check(root))
        if(root is None):
            # print(root.data)
            return self.prev_root
        else:
            # print(root.data)
            if(data.data[level%len(data.data)]<root.data[level%len(data.data)]):
                self.prev_root=root
                tb=self.search(data,root.left,level+1)
            else:
                self.prev_root=root
                tb=self.search(data,root.right,level+1)
        self.qualify(data,tb)
        self.qualify(data,root)
        if(abs(root.data[level%len(data.data)]-data.data[level%len(data.data)])<data.best_dis):
            if(data.data[level%len(data.data)]<root.data[level%len(data.data)]):
                self.prev_root=root
                tb=self.search(data,root.right,level+1)
            else:
                self.prev_root=root
                tb=self.search(data,root.left,level+1)
        elif(len(data.nearest)<data.k):
                self.qualify(data,self.search(data,root.right,level+1))
                self.qualify(data,self.search(data,root.left,level+1))



        self.qualify(data,tb)
        return tb
    # def KNN_protocol(self,data:Node,k):
    #     bests=deque()
    #     best_dist=float('inf')
    #     for i in range(k):
    def nearest(self,data:Node):
        best=self.search(data,self.root,0)
    def knearest(self,data:Node,k):
        self.search(data,self.root,0)
        data.k=k
        alpha=list(data.nearest)
        alpha.sort(key=lambda x: x[-1])
        return alpha[:k]
    def point_exist(self,data:Node):
        if(data.best_dis==float('inf')):
            self.search(data,self.root,0)
        if(data.best_dis==0):
            return True
        else:
            return False
    def rewire(self,root:Node):
        if(root is None):
            return
        else:
            if(root.right is not None):
                root.right.parent=root
                self.rewire(root.right)
            if(root.left is not None):
                root.left.parent=root
                self.rewire(root.left)
    def find_min(self,root:Node,level):
        s=0
        l=0
        best=Node(root.data)
        if(root.left is None and root.right is None):
            return root
        if(root.right is not None):
            s=self.find_min(root.right,level)
            if(s.data[level%len(s.data)]<=best.data[level%len(best.data)]):
                best=Node(s.data)
        if(root.left is not None):
            l=self.find_min(root.left,level)
            if(l.data[level%len(l.data)]<=best.data[level%len(best.data)]):
                best=Node(l.data)
        return best
    def find_range(self):
        b=list(self)
        print(b[0])


    def find_max(self,root:Node,level):
        s=0
        l=0
        best=Node(root.data)
        if(root.left is None and root.right is None):
            return root
        if(root.right is not None):
            s=self.find_max(root.right,level)
            if(s.data[level%len(s.data)]>=best.data[level%len(best.data)]):
                best=Node(s.data)
        if(root.left is not None):
            l=self.find_max(root.left,level)
            if(l.data[level%len(l.data)]>=best.data[level%len(best.data)]):
                best=Node(l.data)
        return best
    def replacement(self,root:Node):
        if(root.right is not None):
            level=root.level
            s=self.find_min(root.right,level)
        elif(root.left is not None):
            level=root.level
            s=self.find_max(root.left,level)
        else:
            s=None
        return s
    def delete_it(self,target:Node):
        if(self.point_exist(target)):
            self.delete_node(self.root,target)
            self.rewire(self.root)
        else:
            print("Doesn't exist #7898")
    def delete_node(self,root:Node,target:Node):
        if(1==1):
            if(root.data==target.data):
                if(root.left is None and root.left is None):
                    if(root.parent.right.data==root.data):
                        root.parent.right=None
                    elif(root.parent.left==root.data):
                        root.parent.left=None
                    root=None
                    return None
                else:
                    s=self.replacement(root)
                    #################################action line
                    if(s is not None):
                        temp=s.data
                        temp1=s.index
                        self.delete_node(root,s)
                        root.data=temp
                        root.index=temp1
                    else:
                        print(root.data)
                        print("#8949-12044")
                        root=None
                        return 0
            elif(root.data[root.level%len(root.data)]>target.data[root.level%len(root.data)]):
                if (root.left is not None):
                    self.delete_node(root.left,target)
                else:
                    print("Impossible#47899")
            elif(root.data[root.level%len(root.data)]<=target.data[root.level%len(root.data)]):
                if(root.right is not None):
                    self.delete_node(root.right,target)
                else:
                    print("#$4550123")
        else:
            print("Doesn't exist!")
    def __iter__(self):
        a=[]
        lev=0
        self.nodes.append((self.root,lev))
        m=0
        k=0
        while(len(self.nodes)):
            b=self.nodes.popleft()
            a.append(k:=self.to_list(b[0],b[1]))
            m=max(m,k[-1])
        k=0
        # lev=[]
        o=[]
        for i in a:
            if(i[1]>k):
                k+=1
                # print()
                # print(i[0] , end=" , ")
            # lev.append(str(tuple(i[0])))
            o.append(tuple(i[0]))
        # o=lev.copy()
        # lev=",".join(lev)
        # print(lev)
        yield o
    def __str__(self) -> str:
        a=[]
        lev=0
        self.nodes.append((self.root,lev))
        m=0
        k=0
        while(len(self.nodes)):
            b=self.nodes.popleft()
            a.append(k:=self.to_list(b[0],b[1]))
            m=max(m,k[-1])
        k=0
        lev=[]
        # o=[]
        for i in a:
            if(i[1]==k):
                print(i[0] , end=" , ")
            elif(i[1]>k):
                k+=1
                print()
                print(i[0] , end=" , ")
            lev.append(str(tuple(i[0])))
            # o.append(tuple(i[0]))
        # o=lev.copy()
        lev=",".join(lev)
        # print(lev)
        return ""
    def to_list(self,root:Node,level):
        if(root is None):
            return ()
        else:
            if(root.left is not None):
                self.nodes.append((root.left,level+1))
            if(root.right is not None):
                self.nodes.append((root.right,level+1))
            return (tuple(root.data),level)
            # return (tuple([root.level,root.data]),level)
    # where to check outputs






    # def find_nearest(self,data:Node):

## Test Cell

In [ ]:
a = [(51,75), (25,40), (70,70), (10,30), (35,52), (55,1),(60,80),(1,10),(50,50)]
b=KDTree(len(a))
b.generate(a)
d=Node((29,40))
d.level=1
k=b.knearest(d,4)
# b.delete_it(d)
# print((b.find_max(b.root.left,1)).data)
print(*k,sep="\n")
# print(d.best_node)
# print(b.point_exist(d))
# print(list(b)[0])
# print(b)
# b.find_range()
# print(b)

(<__main__.Node object at 0x7f6c77db1c70>, 16)
(<__main__.Node object at 0x7f6c77d2e2b0>, 180)
(<__main__.Node object at 0x7f6c77d2efd0>, 461)
(<__main__.Node object at 0x7f6c77d2e100>, 541)


# Copy from drive



In [ ]:
import os
import shutil

In [ ]:

# shutil.copytree("drive/MyDrive/dsproject/","/content/new")

shutil.copytree("drive/MyDrive/dsproject/test.csv","/content")
shutil.copytree("drive/MyDrive/dsproject/test_labels.csv","/content")
shutil.copytree("drive/MyDrive/dsproject/train.csv","/content")
shutil.copytree("drive/MyDrive/dsproject/train_labels.csv","/content")


FileExistsError: ignored

# KNN Classifier

In [88]:
class KNNClassifier:
  def __init__(self,alls:list,k) -> None:   #alls should be tuple not a Node
      self.main_tree=KDTree(k)
      self.main_tree.generate(alls)
      self.dataset=None
      self.labels=None
  def classify(self,data):     #data should be a Node or Not
      if(not isinstance(data,Node)):
          data=Node(data)
      k=5
      alpha=self.main_tree.knearest(data,k)
      beta=self.extract(alpha)
      zeta=[]
      # print(beta)
      for i in beta:
        # print(i)
        # print(labels.iloc[i])
        zeta.append(self.labels.iloc[i])

      zeta=pd.DataFrame(zeta)
      zeta.reset_index(inplace=True)
      zeta.columns=["index","data"]
      # print(zeta)
      return zeta["data"].mode()[0]
  def classify_all(self,datas:list):
      out = display(progress(0, len(datas)), display_id=True)
      out1=display(0,display_id=True)
      

      c=0
      z=[]
      for i in datas:
        c+=1
        ttt=self.classify(i)
        percent=format((c/len(datas))*100,".2f")
        z.append(ttt)
        out1.update(str(percent)+" %")
        out.update(progress(c,len(datas)))
        # print(c, end=" ")
      return pd.Series(z)

  
  def extract(self,alpha:list):
    temp=[]
    for i in alpha:
      temp.append(i[0].index)
    return temp

    




# Read Datasets to Train and Create the Tree


In [89]:
# Train Dataset


train_dataset=pd.read_csv("train.csv")
# test_case=train_dataset.iloc[200:300]
# train_dataset=train_dataset.head(10)

In [90]:
# Labels Dataset
labels_dataset=pd.read_csv("train_labels.csv")

In [91]:
# Convert dataset rows to tuple
final_dataset=train_dataset.iloc[:].apply(lambda x: tuple(x),axis=1)
final_dataset=list(final_dataset)

In [92]:
len(final_dataset[0])

784

In [93]:
# Create the tree
k=784
mytree=KNNClassifier(final_dataset,k)
mytree.dataset=final_dataset
mytree.labels=labels_dataset

# It's Show Time!

In [78]:
test_dataset=pd.read_csv("test.csv")

In [79]:
test_dataset=test_dataset.iloc[:].apply(lambda x: tuple(x),axis=1)
test_dataset=list(test_dataset)

In [80]:
# pd.Series(labels_dataset.head(10)["0"])
test_label_dataset=pd.read_csv("test_labels.csv")
test_label_dataset.columns=["data"]
# test_label_dataset.head()

In [81]:
serie_test_label=pd.Series(test_label_dataset["data"])

In [94]:
res=mytree.classify_all(test_dataset)

'100.00 %'

## Accuracy

In [95]:
# a=[1,2,3,4,5,6,0,8,9,10]
# b=[1,2,3,4,5,1,7,1,2,3]
a=res.copy()
b=serie_test_label.copy()
# b=pd.Series(b)
d=pd.Series(a==b)

p=d.value_counts(True)
type(p)
acc=p.iloc[0]*100

acc

95.97989949748744